In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

# List of folders
folders = [
    "Birds",
    "Graffiti",
    "Objects",
    "Roadkill"
]


    # "Pothole",
    # "Roadkill",

# Root path where your folders are located
root_dir = '/content/drive/MyDrive/SJHacks Hackathon/Datasets/'

# Paths for the combined images and labels directories
combined_images_dir = '/content/drive/MyDrive/FinalCombined/Images'
combined_labels_dir = '/content/drive/MyDrive/FinalCombined/Labels'

# Make directories if they don't exist
os.makedirs(combined_images_dir, exist_ok=True)
os.makedirs(combined_labels_dir, exist_ok=True)

# Create a mapping for class labels
class_mapping = {folder: index for index, folder in enumerate(folders)}

# Loop over each folder to collect images and annotations
for folder in folders:
    print(folder)
    images_dir = os.path.join(root_dir, folder, 'images')
    labels_dir = os.path.join(root_dir, folder, 'bounding_box')

    for image_name in os.listdir(images_dir):
        if image_name.endswith('.jpg'):
            # Get the image path
            image_path = os.path.join(images_dir, image_name)

            # Copy the image to the combined images directory
            new_image_name = f"{folder}_{image_name}"
            new_image_path = os.path.join(combined_images_dir, new_image_name)
            shutil.copy(image_path, new_image_path)


Birds
Graffiti
Objects
Roadkill


In [ ]:

import os

# List of folders
folders = [
    "Birds",
    "Graffiti",
    "Objects",
    "Pothole",
    "Roadkill",
    "Weeds"
]

# Root path where your folders are located
root_dir = '/content/drive/MyDrive/SJHacks Hackathon/Datasets/'

# Paths for the combined images and labels directories
combined_labels_dir = '/content/drive/MyDrive/FinalCombined/Labels'

# Create a mapping for class labels
class_mapping = {folder: index for index, folder in enumerate(folders)}

# Loop over each folder to adjust the labels
for folder in folders:
    labels_dir = os.path.join(root_dir, folder, 'bounding_box')

    for label_name in os.listdir(labels_dir):
        if label_name.endswith('.txt'):
            label_path = os.path.join(labels_dir, label_name)

            # Read the annotations from the label file
            with open(label_path, 'r') as label_file:
                annotations = label_file.readlines()

            # Write the adjusted annotations to the new label file
            new_label_path = os.path.join(combined_labels_dir, f"{folder}_{label_name}")
            with open(new_label_path, 'w') as new_label_file:
                for annotation in annotations:
                    parts = annotation.strip().split()
                    # Adjust the class index based on the folder it came from
                    parts[0] = str(class_mapping[folder])
                    new_label_file.write(" ".join(parts) + '\n')


In [ ]:
import os
import shutil
import random
import re
from sklearn.model_selection import train_test_split

# Define paths
combined_images_dir = '/content/drive/MyDrive/FinalCombined/Images'
combined_labels_dir = '/content/drive/MyDrive/FinalCombined/Labels'

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

# Paths for the split datasets
train_images_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Train/Images'
train_labels_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Train/Labels'

val_images_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Val/Images'
val_labels_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Val/Labels'

test_images_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Test/Images'
test_labels_dir = '/content/drive/MyDrive/SJHacks Hackathon/Spliting/Test/Labels'

# Create directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# List all images and their corresponding labels
image_files = [f for f in os.listdir(combined_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Create a dictionary to store images by their class
images_by_class = {i: [] for i in range(6)}  # Assuming 6 classes, update if necessary

# Class name mapping (Ensure no extra spaces)
class_name_mapping = {
    'Birds': 0,
    'Graffiti': 1,
    'Objects': 2,
    'Pothole': 3,
    'Roadkill': 4,
    'Weeds': 5
}

# Debug: Check if the images are correctly classified into classes
for image_file in image_files:
    # Extract the class label from the image file name
    #class_name = image_file.split('_')[0]
    class_name = image_file.split('_')[0].strip()


    if class_name not in class_name_mapping:
        print(f"Warning: {class_name} not found in class mappings for image {image_file}")
        continue

    class_index = class_name_mapping[class_name]
    images_by_class[class_index].append(image_file)

# Debug: Print class distribution
print("Class distribution:")
for class_index, class_images in images_by_class.items():
    print(f"Class {class_index}: {len(class_images)} images")

# Perform the split for each class
train_files = []
val_files = []
test_files = []

for class_index, class_images in images_by_class.items():
    if len(class_images) == 0:
        print(f"Warning: No images found for class {class_index}. Skipping this class.")
        continue

    train, temp = train_test_split(class_images, train_size=train_ratio, random_state=42)
    val, test = train_test_split(temp, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)

    train_files.extend(train)
    val_files.extend(val)
    test_files.extend(test)

# --- START NEW CLEANING + MOVING FUNCTION ---

def clean_filename(name):
    """Remove ' (1)', ' (2)' patterns from filename."""
    return re.sub(r' \(\d+\)', '', name)

def move_files(file_list, source_dir, target_image_dir, target_label_dir):
    for file_name in file_list:
        # Move image file
        image_path = os.path.join(source_dir, file_name)

        if not os.path.exists(image_path):
            print(f"Warning: Image file {file_name} not found. Skipping.")
            continue

        shutil.copy(image_path, os.path.join(target_image_dir, file_name))

        # Prepare label file
        clean_name = clean_filename(file_name)
        label_name = os.path.splitext(clean_name)[0] + '.txt'
        label_path = os.path.join(combined_labels_dir, label_name)

        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(target_label_dir, label_name))
        else:
            print(f"Warning: Label file {label_name} for image {file_name} not found. Skipping.")

# --- END FUNCTION ---

# Move images and labels to the corresponding directories
move_files(train_files, combined_images_dir, train_images_dir, train_labels_dir)
move_files(val_files, combined_images_dir, val_images_dir, val_labels_dir)
move_files(test_files, combined_images_dir, test_images_dir, test_labels_dir)

print(f"Train set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")
print(f"Test set: {len(test_files)} images")


Class distribution:
Class 0: 3993 images
Class 1: 965 images
Class 2: 521 images
Class 3: 665 images
Class 4: 89 images
Class 5: 1310 images
Train set: 5277 images
Validation set: 753 images
Test set: 1513 images


In [ ]:
# Function to calculate and print the class distribution
def print_class_distribution(file_list, class_name_mapping):
    class_distribution = {class_index: 0 for class_index in range(len(class_name_mapping))}

    for file_name in file_list:
        # Extract the class label from the image file name
        class_name = file_name.split('_')[0]
        class_index = class_name_mapping.get(class_name)
        if class_index is not None:
            class_distribution[class_index] += 1

    return class_distribution

# Print class distribution for each set
print("Class distribution in the Train set:")
train_class_distribution = print_class_distribution(train_files, class_name_mapping)
for class_index, count in train_class_distribution.items():
    print(f"Class {class_index} ({list(class_name_mapping.keys())[class_index]}): {count} images")

print("\nClass distribution in the Validation set:")
val_class_distribution = print_class_distribution(val_files, class_name_mapping)
for class_index, count in val_class_distribution.items():
    print(f"Class {class_index} ({list(class_name_mapping.keys())[class_index]}): {count} images")

print("\nClass distribution in the Test set:")
test_class_distribution = print_class_distribution(test_files, class_name_mapping)
for class_index, count in test_class_distribution.items():
    print(f"Class {class_index} ({list(class_name_mapping.keys())[class_index]}): {count} images")


Class distribution in the Train set:
Class 0 (Birds): 2795 images
Class 1 (Graffiti): 675 images
Class 2 (Objects): 364 images
Class 3 (Pothole): 465 images
Class 4 (Roadkill): 62 images
Class 5 (Weeds): 916 images

Class distribution in the Validation set:
Class 0 (Birds): 399 images
Class 1 (Graffiti): 96 images
Class 2 (Objects): 52 images
Class 3 (Pothole): 66 images
Class 4 (Roadkill): 9 images
Class 5 (Weeds): 131 images

Class distribution in the Test set:
Class 0 (Birds): 799 images
Class 1 (Graffiti): 194 images
Class 2 (Objects): 105 images
Class 3 (Pothole): 134 images
Class 4 (Roadkill): 18 images
Class 5 (Weeds): 263 images
